## Alice's Adventures in <font color='green'>Word</font>land

## Alice's Adventures in <b style='color:green'>Word</b>land

<p style='color:red'>This is some red text.</p> <br>
<font color="red">This is some text!</font> <br>
These are <b style='color:red'>red words</b>. <br>


## Alice's Adventures in "Word"land

![my_image](./aliceimage.png)

Alice likes playing a word game. If you give her a word, she will give a suggestion that she thinks it's closest to the word from the vocabulary she knows. Her vocabulary is limited to words from story of "Alice's Adventures in Wonderland". 

When she knows the word you gave her, she will give the same word as suggestion. On the other hand, if she doesn't know, she will find the "cloesest word" based on the following..
- among words in her vocabulary that the lengh of them are +/- 1 letter away 
- ignore any punctuations such as comma(,), period(.), colon(:), exclamation point(!), etc.
- a one that has biggiest number of matching letter 
- if there are two or more words that has same number of matching letters, pick a one that has more occurence in the story
             
          [ for example ]
            Let's say you gave her a word "cha" for a word suggestion. 
            In her vocab, 2~4 letters long words are, hypothetically, act, been, chai, chip, hhhh, and map.
              
            If we compare "cha" to "act", two letters in cha exist in act: 'c' and 'a'.
            Therefore:  cha --> act(2)
                            --> been(0)
                            --> chai(3)
                            --> chip(2)
                            --> hhhh(1)
                            --> map(1)
                                    ↳ numbers in parenthesis shows the number of matching letters.

   - Note that we are checking whether each letters in your word can be found in Alice's vocab. This reduce the chance of the getting not-so-ideal suggestion from Alice. If we were counting the number of matching letter in Alice's word to your word, the number of matching letters of "hhhh" is 4 instead of 1.
            
==============

The last two from Alice's approach of finding the closest word, would they found you the best/ultimate suggestion? How could we define the "ultimate" suggestion? What would you like to get as a suggested word between "that" or "the" when you give her "tha". Alice will give you "that" since there are 3 matching letters. However, "the" has a higher probability since we use it more often than "that" in sentences. On the other hand, it's possible that user prematually pressed the enter before finishing typing each letters of "that". How should we decide the precedence of these two steps......


- (Of course there might be better soulution. We will discuss it further later.)
- (Will improve it later, for now we will keep it simple without seeking for ultimate solution.)

==============



_This is similar to word auto-correction but assuming that the english words in this world is limited to those word from the story._
<br>
<br>
<br>


text file from here: https://archive.org/stream/alicesadventures19033gut/19033.txt <br>
image from here: https://blog.whsmith.co.uk/alices-adventures-in-wonderland-free-colouring-downloads/

#### Let's first construct Alice's vocabulary from Alice's Adventures in Wonderland story 

In [2]:
def txt_to_dict(text_file):
    """    
    Takes a text file and put each setence on to a list in a lower case without any punctuations.
    Then, it creates a dictionary that takes each word from the sentences as a key with its value.   
    
    INPUT : text file
    OUTPUT: a dictionary, {'word' : [length of word, occurrence]}
    """

    import re

    ## Opening the text file
    with open(text_file) as alice_story:  
        
        ## [List] create one big list which each items are a single sentence, separated by \n but doesn't contain \n
        story_lines = alice_story.read().splitlines() # length: 3736
        
        ## [List] put each sentences into a one new big list after remove speical_char and convert them to lowercase
        cleaned_lines = []                            # length: 3736
        for sentence in story_lines:
            cleaned_lines.append(re.sub('[^A-Za-z0-9 ]+', '', sentence).lower()) # keeping whitespaces
         
        ## {Dictionary} split into single word and put it on to a dictionary
        words_dict = {}                               # length: 3254
        for sentence in cleaned_lines:
            ## [List] "words" is a list with words from one sentence, there will be 3736 lists
            words = sentence.split(" ")  
            for w in words:
                if w in words_dict: # when a word already exists in words_dict, just increment the frequency 
                    words_dict[w][1] += 1
                else:               # add a new entry in words_dict with word length and freq info
                    words_dict[w] = [len(w), 1]
                    
        print (words_dict)        
        return words_dict

#### Below is how Alice's vocabulary list going to be look like after the manipulation to the text file.

In [3]:
alice_dict = txt_to_dict("aliceText.txt")

{'project': [7, 87], 'gutenbergs': [10, 2], 'alices': [6, 17], 'adventures': [10, 11], 'in': [2, 428], 'wonderland': [10, 8], 'by': [2, 76], 'lewis': [5, 4], 'carroll': [7, 4], '': [0, 2319], 'this': [4, 181], 'ebook': [5, 9], 'is': [2, 128], 'for': [3, 179], 'the': [3, 1804], 'use': [3, 29], 'of': [2, 625], 'anyone': [6, 5], 'anywhere': [8, 3], 'at': [2, 224], 'no': [2, 97], 'cost': [4, 4], 'and': [3, 912], 'with': [4, 228], 'almost': [6, 8], 'restrictions': [12, 2], 'whatsoever': [10, 2], 'you': [3, 429], 'may': [3, 28], 'copy': [4, 12], 'it': [2, 541], 'give': [4, 16], 'away': [4, 27], 'or': [2, 154], 'reuse': [5, 2], 'under': [5, 22], 'terms': [5, 22], 'gutenberg': [9, 28], 'license': [7, 15], 'included': [8, 3], 'online': [6, 4], 'wwwgutenbergorg': [15, 3], 'title': [5, 1], 'author': [6, 1], 'posting': [7, 1], 'date': [4, 4], 'june': [4, 1], '25': [2, 1], '2008': [4, 1], '11': [2, 1], 'release': [7, 1], 'march': [5, 34], '1994': [4, 1], 'last': [4, 34], 'updated': [7, 2], 'october

#### Now she needs set up a procedure to look up a "cloest word" from her vocab list.

In [4]:
def max_match(words_dict, user_word):
    """
    Counts the number of matching letters for every words in length of +/- 1 of user_word in a words_dict 
    compared to user_word.
    
    INPUT : a dictionary, {'word' : [length of word, occurrence]}
    OUTPUT: a string, 'word'    
    """
    
    closest_word = ""
    closest_match = 0

    for w in words_dict:
        ## reset the value of matching_char for every word in words_dict
        matching_char = 0
        proper_length = False     # for dubugging 
        ## pick a word in words_dict 1 letter shorter or longer than the user_word
        if (len(user_word) -1) <= words_dict[w][0] <= (len(user_word) + 1):
            proper_length = True  # for dubugging 
            ## check whether each letters in user_word exists in a word in words_dict
            for i in range(len(user_word)):
                if user_word[i] in w:
                    matching_char += 1
                    
            ## if we were doing like this instead of above, 
            ## this counts the number of matching letter in a word in words_dict to user_word
            ## as discussed at the beginning, this draws not-so-ideal results
#             for i in range(len(w)):
#                 if w[i] in user_word:
#                     matching_char += 1
        
        ## update closest_match and closest_word if applicable
        if matching_char > closest_match:
            closest_match = matching_char
            closest_word = w
        ## if new candidate word's matching_char is same as current closest_match, select the higher freq one
        elif matching_char == closest_match: 
            if words_dict[w][1] > words_dict[closest_word][1]:
                closest_word = w
                
        ## dubugging purpose         
#         print ("tested:", w, words_dict[w], proper_length, " -----  status:", closest_match, closest_word)

    return closest_word    

#### Let's play word suggestion game with Alice.

In [72]:
search_word = input("Which word would you like to ask her? ")
print ("Alice here, how about '%s' instead of '%s'?" % (max_match(alice_dict, search_word), search_word))

Which word would you like to ask her? tha
Alice here, how about 'that' instead of 'tha'?


In [5]:
search_word = input("Which word would you like to ask her? ")
print ("Alice here, how about '%s' instead of '%s'?" % (max_match(alice_dict, search_word), search_word))

Which word would you like to ask her? alic
Alice here, how about 'alice' instead of 'alic'?


#### Result of below shows her journey to that conclusion. 

In [75]:
search_word = "tha"
max_match(alice_dict, search_word)

tested: project [7, 87] False  -----  status: 0 
tested: gutenbergs [10, 2] False  -----  status: 0 
tested: alices [6, 17] False  -----  status: 0 
tested: adventures [10, 11] False  -----  status: 0 
tested: in [2, 428] True  -----  status: 0 
tested: wonderland [10, 8] False  -----  status: 0 
tested: by [2, 76] True  -----  status: 0 
tested: lewis [5, 4] False  -----  status: 0 
tested: carroll [7, 4] False  -----  status: 0 
tested:  [0, 2319] False  -----  status: 0 
tested: this [4, 181] True  -----  status: 2 this
tested: ebook [5, 9] False  -----  status: 2 this
tested: is [2, 128] True  -----  status: 2 this
tested: for [3, 179] True  -----  status: 2 this
tested: the [3, 1804] True  -----  status: 2 the
tested: use [3, 29] True  -----  status: 2 the
tested: of [2, 625] True  -----  status: 2 the
tested: anyone [6, 5] False  -----  status: 2 the
tested: anywhere [8, 3] False  -----  status: 2 the
tested: at [2, 224] True  -----  status: 2 the
tested: no [2, 97] True  -----  

'that'

This works decent for 1 distance away words .
But propject --> pepper becuase there are more "p" and thereforem higher matchign_char score. 
to fix this error.. I will use different method



if user types "tha"
the options are the, than, that... 
since the most occuring one is the, it will return "the"

flaw:
projec --> project
proeec --> pepper



   - Since she counts only the number of matching letter, regardless each letter's location within a word nor the number of presence of each letter(i.e. how many 'h' in the given word), this has flaw of getting "hhhh" as a closest word of "cha". 

In [98]:
# ????how to access only the story part???? ????
# biginning is line 40th


import re

alice_story = open("aliceText.txt", "r")
alice_story.seek(0)

story_lines = alice_story.read().splitlines() 
# print ("story_lines len", len(story_lines))



cleaned_lines len 3736
['Project Gutenbergs Alices Adventures in Wonderland by Lewis Carroll', '', 'This eBook is for the use of anyone anywhere at no cost and with', 'almost no restrictions whatsoever  You may copy it give it away or', 'reuse it under the terms of the Project Gutenberg License included', 'with this eBook or online at wwwgutenbergorg', '', '', 'Title Alices Adventures in Wonderland', '', 'Author Lewis Carroll', '', 'Posting Date June 25 2008 EBook 11', 'Release Date March 1994', 'Last Updated October 6 2016', '', 'Language English', '', 'Character set encoding UTF8', '', ' START OF THIS PROJECT GUTENBERG EBOOK ALICES ADVENTURES IN WONDERLAND ', '', '', '', '', '', '', '', '', '', '', 'ALICES ADVENTURES IN WONDERLAND', '', 'Lewis Carroll', '', 'THE MILLENNIUM FULCRUM EDITION 30', '', '', '', '', 'CHAPTER I Down the RabbitHole', '', 'Alice was beginning to get very tired of sitting by her sister on the', 'bank and of having nothing to do once or twice she had peeped into